In [1]:
import json
import datetime
import pickle
from lxml import html
from bs4 import BeautifulSoup
import urllib.request
import requests
import re

In [2]:
FLcertificationFile = "./FL/FLCertifications.p"
FLcriminalFile = "./FL/FLCriminal.p"
FLdisciplinaryFile = "./FL/FLDisciplinary.p"
FLfacultyFile = "./FL/FLFaculty.p"
FLmembershipsFile = "./FL/FLMemberships.p"
FLPostgradFile = "./FL/FLPostGrad.p"
FLpublicationsFile = "./FL/FLPublications.p"
FLLanguagesFile = "./FL/FLLanguages.p"
FLOtherMedFile = "./FL/FLOtherMed.p"
FLAwardsFile = "./FL/FLAwards.p"
FLHospitalPrivFile = "./FL/FLHospitalPriv.p"

In [3]:
FLcertifications = pickle.load (open(FLcertificationFile, "rb"))
FLcriminal = pickle.load (open(FLcriminalFile, "rb"))
FLdisciplinary = pickle.load (open(FLdisciplinaryFile, "rb"))
FLfaculty = pickle.load (open(FLfacultyFile, "rb"))
FLmemberships = pickle.load (open(FLmembershipsFile, "rb"))
FLPostgrad = pickle.load (open(FLPostgradFile, "rb"))
FLpublications = pickle.load (open(FLpublicationsFile, "rb"))
FLLanguages = pickle.load (open(FLLanguagesFile, "rb"))
FLOtherMed = pickle.load (open(FLOtherMedFile, "rb"))
FLAwards = pickle.load (open(FLAwardsFile, "rb"))
FLHospitalPriv = pickle.load (open(FLHospitalPrivFile, "rb"))

CAdisciplinary = pickle.load (open("CADisciplinary-Copy1.p", "rb"))

In [14]:
the_data = []
with open("xae-Copy1", 'r') as f:
    for line in f:
        try:
            line = json.loads(line)
            the_data.append(line)
        except ValueError:
            pass

FileNotFoundError: [Errno 2] No such file or directory: 'xae-Copy1'

In [9]:
len(the_data)

916190

In [13]:
a = datetime.datetime.now().replace(microsecond=0)
counter = 0
with open("xad1.jsonl", 'a') as output:
    for l in the_data[600000:]:
        #l = json.loads(l)
        l['other_med_schools'] = None
        l['postgrad_training'] = None
        l['board_certs'] = None
        l['hospital_privileges'] = None
        l['disciplinary_history'] = None
        l['privileges_revoked'] = None
        l['criminal_history'] = None
        l['malpractice_awards'] = None
        l['malpractice_settlements'] = None
        l['publications'] = None
        l['organizations'] = None
        l['awards'] = None
        l['languages'] = None
        l['faculty_appts'] = None
        if 'degree' in l:
            try:
                if l['degree'] == 'MD':
                    if 'licenses' in l:
                        for i in range(len(l['licenses'])):
                            if 'state' in l['licenses'][i]:
                                if l['licenses'][i]['state'] == 'CA':
                                    #extracts CA license number from data
                                    if 'number' in l['licenses'][i]:
                                        licenseID = l['licenses'][i]['number']
                                        l['disciplinary_history'] = CAdisciplinary.get(licenseID)
                                        for k in range(len(l['specialty_primary'])):
                                            if l['specialty_primary'][k] == 'Acupuncturist':
                                                ID = re.findall('\d+', licenseID)
                                                for j in range (len(ID)):
                                                    l['disciplinary_history'] = CAAcupuncture(ID[j])
                                if l['licenses'][i]['state'] == 'NY':
                                    #extracts NY license number from data
                                    if 'number' in l['licenses'][i]:
                                        licenseID = l['licenses'][i]['number']
                                        ID = re.findall('\d+', licenseID)
                                        for j in range (len(ID)):
                                            l['disciplinary_history'] = NYdisciplinary(ID[j])
                                if l['licenses'][i]['state'] == 'GA':
                                    #extracts GA license number from data
                                    if 'number' in l['licenses'][i]:
                                        licenseID = l['licenses'][i]['number']
                                        ID = re.findall('\d+', licenseID)
                                        ##if ID is list:
                                        for j in range (len(ID)):
                                            websiteBase = 'http://www.gaphysicianprofile.org/profile.ShowProfileAction.action?lic_nbr='
                                            website = websiteBase + ID[j]
                                            page = urllib.request.urlopen(website)
                                            soup = BeautifulSoup(page, "lxml")
                                            ##information is stored in tables
                                            all_tables=soup.find_all('table')
                                            if len(all_tables) > 4:
                                                print('GA idx = {}'.format(counter))
                                                l['med_school'] = medSchool()
                                                l['other_med_schools'] = otherMedSchool()
                                                l['postgrad_training'] = postgradTraining()
                                                l['board_certs'] = boardCerts()
                                                l['hospital_privileges'] = hospitalPrivileges()
                                                l['disciplinary_history'] = disciplinary()
                                                l['privileges_revoked'] = privilegeRevocation()
                                                l['criminal_history'] = criminalOffenses()
                                                l['malpractice_awards'] = malpracticeAwards()
                                                l['malpractice_settlements'] = malpracticeSettlements()
                                                l['publications'] = Optionals()[0]
                                                l['organizations'] = Optionals()[1]
                                                l['awards'] = Optionals()[2]
                                                l['languages'] = Optionals()[3]
                                                l['faculty_appts'] = Optionals()[4]
                                if l['licenses'][i]['state'] == 'FL':
                                    if 'number' in l['licenses'][i]:
                                        licenseID = l['licenses'][i]['number']
                                        ID = re.findall('\d+', licenseID)
                                        for j in range (len(ID)):
                                            #print('FL idx = {}'.format(counter))
                                            l['other_med_schools'] = FLOtherMed.get(ID[j])
                                            l['postgrad_training'] = FLPostgrad.get(ID[j])
                                            l['board_certs'] = FLcertifications.get(ID[j])
                                            l['disciplinary_history'] = FLdisciplinary.get(ID[j])
                                            l['awards'] = FLAwards.get(ID[j])
                                            l['postgrad_training'] = FLPostgrad.get(ID[j])
                                            l['criminal_history'] = FLcriminal.get(ID[j])
                                            l['organizations'] = FLmemberships.get(ID[j])
                                            l['publications'] = FLpublications.get(ID[j])
                                            l['hospital_privileges'] = FLHospitalPriv.get(ID[j])
                                            l['languages'] = FLLanguages.get(ID[j])
                                            l['faculty_appts'] = FLfaculty.get(ID[j])
            
                for key in list(l.keys()):
                    if type(l[key]) == list:
                        if len(l[key]) == 0:
                            l[key] = None
            except IndexError:
                pass
        counter +=1
        json.dump(l, output)
        output.write('\n')
        
b = datetime.datetime.now().replace(microsecond=0)
print(b-a)

GA idx = 25
GA idx = 87
GA idx = 238
GA idx = 295
GA idx = 295
GA idx = 460
GA idx = 468
GA idx = 545
GA idx = 1108
GA idx = 1318
GA idx = 1366
GA idx = 1463
GA idx = 1671
GA idx = 1678
GA idx = 1806
GA idx = 2385
GA idx = 2415
GA idx = 2495
GA idx = 3430
GA idx = 3505
GA idx = 3718
GA idx = 3880
GA idx = 4800
GA idx = 4894
GA idx = 4894
GA idx = 4894
GA idx = 4895
GA idx = 5492
GA idx = 5492
GA idx = 5791
GA idx = 5890
GA idx = 6394
GA idx = 6852
GA idx = 6852
GA idx = 7118
GA idx = 7326
GA idx = 7506
GA idx = 7664
GA idx = 7697
GA idx = 8178
GA idx = 8188
GA idx = 8189
GA idx = 8189
GA idx = 8289
GA idx = 8547
GA idx = 8611
GA idx = 8684
GA idx = 8690
GA idx = 8690
GA idx = 8783
GA idx = 9068
GA idx = 9242
GA idx = 9538
GA idx = 9649
GA idx = 9752
GA idx = 9798
GA idx = 9798
GA idx = 9798
GA idx = 9881
GA idx = 9987
GA idx = 10021
GA idx = 10186
GA idx = 10196
GA idx = 10316
GA idx = 10348
GA idx = 10352
GA idx = 10439
GA idx = 10439
GA idx = 10489
GA idx = 10490
GA idx = 10490
GA id

In [6]:
##Georgia Licenses
licenseID = '034779'
websiteBase = 'http://www.gaphysicianprofile.org/profile.ShowProfileAction.action?lic_nbr='
website = websiteBase + licenseID

page = urllib.request.urlopen(website)
soup = BeautifulSoup(page, "lxml")
##information is stored in tables
all_tables=soup.find_all('table')

##MEDICAL SCHOOL DICT
def medSchool():
    schoolTables = all_tables[14].find_all('table')
    #should be 7
    numTables = len(schoolTables)
    ##Medical School is always at schoolTables[2]
    if len(schoolTables[2].find_all('tr')) > 2:
        Med1header = []
        Med1info = []
        numM1Schools = len(schoolTables[2].find_all('tr')) - 2
        Med1 = []
        for i in range(len(schoolTables[2].find_all('tr')[1].find_all('td'))):
            try:
                if len(schoolTables[2].find_all('tr')[1].find_all('td')[i].b.contents) > 1:
                    Med1header.append(schoolTables[2].find_all('tr')[1].find_all('td')[i].b.contents[0])
                else:
                    Med1header.append(schoolTables[2].find_all('tr')[1].find_all('td')[i].b.string)
            except:
                TypeError        
        for i in range(2, numM1Schools + 2):
            for j in range(len(schoolTables[2].find_all('tr')[i].find_all('td'))):
                Med1info.append(schoolTables[2].find_all('tr')[i].find_all('td')[j].string)

        for i in range(numM1Schools):
            Med1dict = {}
            for j in range(len(Med1header)):
                Med1dict[Med1header[j]] = Med1info[j + len(Med1header)*i]
            Med1.append(Med1dict)
    else:
        Med1 = None
    return Med1

##Other Med Schools are always at schoolTables[4]  
def otherMedSchool():
    schoolTables = all_tables[14].find_all('table')
    if len(schoolTables[4].find_all('tr')) > 2:
        Med2header = []
        Med2info = []
        numM2Schools = len(schoolTables[4].find_all('tr')) - 2
        Med2 = []
        for i in range(len(schoolTables[4].find_all('tr')[1].find_all('td'))):
            try:
                if len(schoolTables[4].find_all('tr')[1].find_all('td')[i].b.contents) > 1:
                    Med2header.append(schoolTables[4].find_all('tr')[1].find_all('td')[i].b.contents[0])
                else:
                    Med2header.append(schoolTables[4].find_all('tr')[1].find_all('td')[i].b.string)
            except:
                TypeError         
        for i in range(2, numM2Schools + 2):
            for j in range(len(schoolTables[4].find_all('tr')[i].find_all('td'))):
                Med2info.append(schoolTables[4].find_all('tr')[i].find_all('td')[j].string)  

        for i in range(numM2Schools):
            Med2dict = {}
            for j in range(len(Med2header)):
                Med2dict[Med2header[j]] = Med2info[j + len(Med2header)*i]
            Med2.append(Med2dict)
    else:
        Med2 = None   
    return Med2

##Postgrad training is always at schoolTables[6] 
def postgradTraining():
    schoolTables = all_tables[14].find_all('table')
    if len(schoolTables[6].find_all('tr')) > 2:
        Med3header = []
        Med3info = []
        numM3Schools = len(schoolTables[6].find_all('tr')) - 2
        Med3 = []
        Med3header.append(schoolTables[6].find_all('tr')[0].td.b.string)
        for i in range(len(schoolTables[6].find_all('tr')[1].find_all('td'))):
            try:
                if len(schoolTables[6].find_all('tr')[1].find_all('td')[i].b.contents) > 1:
                    Med3header.append(schoolTables[6].find_all('tr')[1].find_all('td')[i].b.contents[0])
                else:
                    Med3header.append(schoolTables[6].find_all('tr')[1].find_all('td')[i].b.string)
            except:
                TypeError       
        for i in range(2, numM3Schools + 2):
            for j in range(len(schoolTables[6].find_all('tr')[i].find_all('td'))):
                Med3info.append(schoolTables[6].find_all('tr')[i].find_all('td')[j].string) 

        for i in range(numM3Schools):
            Med3dict = {}
            for j in range(len(Med3header)):
                Med3dict[Med3header[j]] = Med3info[j + len(Med3header)*i]
            Med3.append(Med3dict)
    else:
        Med3 = None 
    return Med3

##make board certification dict 
def boardCerts():
    certTable = all_tables[25]
    if len(certTable.find_all('td')) > 1:
        numCerts = len(certTable.find_all('div'))

        #generate header: CERTIFYING BOARD, SPECIALTY/SUBSPECIALTY
        header = []
        header.append(certTable.find_all('td')[1].string)
        header.append(certTable.find_all('td')[3].string)

        ##how many entries are in postgrad dict
        dictSize = len(header) 

        BoardCertifications = []

        for i in range(numCerts):
            BoardCertifications.append(certTable.find_all('div')[i].string.strip())
            BoardCertifications.append(certTable.find_all('td')[len(certTable.find_all('td')) - 4*i -1].string)

        Boardcert = []
        for i in range(numCerts):
            boardcert = {}
            for j in range(dictSize):
                boardcert[header[j]] = BoardCertifications[j+i*2]
            Boardcert.append(boardcert)
    else:
        Boardcert = None
    return Boardcert

##make hospital privileges dict 
def hospitalPrivileges():
    hospitalTable = all_tables[28]
    if len(hospitalTable.find_all('td')) > 2:
        hospitalTableSize = len(hospitalTable)
        numHospitals = (len(hospitalTable.find_all('td'))-2) // 2

        #generate header: HOSPITAL
        header = []
        header.append(hospitalTable.find_all('td')[2].string)

        ##how many entries are in hospital dict
        dictSize = len(header) 

        HospitalPrivileges = []

        for i in range(numHospitals):
            HospitalPrivileges.append(hospitalTable.find_all('td')[len(hospitalTable.find_all('td'))-1 - 2*i].string.strip())

        Hospitalpriv = []
        for i in range(numHospitals):
            hospitalpriv = {}
            hospitalpriv['Hospital'] = HospitalPrivileges[i]
            Hospitalpriv.append(hospitalpriv)
    else:
        Hospitalpriv = None
    return Hospitalpriv

##make disciplinary history dict 
def disciplinary():
    discpTable = all_tables[32].find_all('table')
    discpTableSize = len(discpTable)
    if discpTableSize > 1:
        Disciplinary = 'YES'
    else:
        Disciplinary = None
    return Disciplinary

##privilege revocation at 36
def privilegeRevocation():
    revocationTable = all_tables[36].find_all('table')
    if len(revocationTable) > 2:
        Revocations = 'YES'
    else:
        Revocations = None
    return Revocations

##criminal offenses at 39
def criminalOffenses():
    criminalTable = all_tables[39].find_all('table')
    if len(all_tables[39].find_all('table')) > 1:
        criminalOffenses = 'YES'
    else:
        criminalOffenses = None
    return criminalOffenses

##Medical malpractice awards 44
def malpracticeAwards():
    malpracticeAwardTable = all_tables[44].find_all('tr')
    if len(malpracticeAwardTable) > 1:
        malpracticeAwards = 'YES'
    else:
        malpracticeAwards = None
    return malpracticeAwards   

##Medical malpractice awards 44
def malpracticeSettlements():
    SettlementTable = all_tables[49].find_all('td')
    if len(SettlementTable) > 1:
        Settlements = 'YES'
    else:
        Settlements = None
    return Settlements 

##Optional information
def Optionals():
    publications = []
    organizations = []
    awards = []
    languages = []
    facultyAppts = []
    
    optTable = all_tables[52].find_all('table')
    ##should be 10 cells
    offset = len(optTable)-10
    
    for i in range (1+offset, len (optTable), 2):
        for j in range(1, len(optTable[i].find_all('tr'))):
            if len(optTable[i].find_all('tr')) > 1:
                val = optTable[i].find_all('tr')[j].td
                if i == 1+offset:
                    publications.append(val.div.string.strip())
                if i == 3+offset:         
                    organizations = val.string.split(',')
                if i == 5+offset:
                    for j in range(1,len(optTable[i].find_all('tr'))):
                        awards.append(optTable[i].find_all('tr')[j].find_all('td')[0].div.string.strip())
                if i == 7+offset:
                    languages = val.div.string.strip().split(',')
                if i == 9+offset:
                    facultyAppts.append(val.div.string.strip())
    
    return publications, organizations, awards, languages, facultyAppts

In [7]:
def NYdisciplinary(licID):
    pt1 = "https://w3.health.ny.gov/opmc/factions.nsf/ByPhysician?SearchView&Query=(FIELD+WebLicense="
    licID = str(licID)
    pt2 = ")&SearchOrder=1&SearchMax=0&SearchWV=TRUE&SearchThesaurus=FALSE"
    url = pt1 + licID + pt2
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    all_tables=soup.find_all('table')
    if len(all_tables) > 1:
        disciplinary = 'YES'
    else:
        disciplinary = None
    return disciplinary

In [8]:
def CAAcupuncture(licID):
    pt1 = 'http://www2.dca.ca.gov/pls/wllpub/WLLQRYNA$LCEV2.QueryView?P_LICENSE_NUMBER='
    pt2 = '&P_LTE_ID=723'

    url = pt1 + licID + pt2

    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page, "lxml")
    all_tables=soup.find_all('table')
    
    if len(all_tables) > 0:
        valTable = all_tables[0].find_all('tr')
        discpAction = valTable[len(valTable) - 1].find_all('td')[1].string
        if discpAction != 'No':
            return discpAction